In [1]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')

In [ ]:
%pip install faiss-cpu --quiet
%pip install google.cloud --quiet
%pip install cloud-sql-python-connector --quiet
%pip install pg8000 --quiet
%pip install langchain-google-vertexai

In [2]:
import numpy as np
import faiss
from pandas import DataFrame
from datetime import datetime

In [3]:
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = 'us-central1'
PROJECT_ID

'cdii-poc'

# Query and Embeddings in PostgreSQL

## Initialise

In [4]:
%load_ext autoreload
%autoreload 2
from nl2sql_src.nl2sql_query_embeddings import PgSqlEmb

# Make sure you update the following variables in-line with your project
PGPROJ = "cdii-poc"
PGLOCATION = 'us-central1'
PGINSTANCE = "cdii-demo-temp"
PGDB = "demodbcdii"
PGUSER = "postgres"
PGPWD = "cdii-demo"
# INDX_FILE = 'saved_index_pgdata' # optoinal, considered as default

pge = PgSqlEmb(PGPROJ, PGLOCATION, PGINSTANCE, PGDB, PGUSER, PGPWD)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
class Initiated


## Create Table

In [ ]:
# Create Table in PostgresDB if not already existing
pge.create_table()

['cdii-poc:us-central1:cdii-demo-temp']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "/home/avijendra/ey/lib/python3.11/site-packages/google/cloud/sql/connector/instance.py", line 298, in _refresh_task
    refresh_data = await refresh_task
                   ^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/ey/lib/python3.11/site-packages/google/cloud/sql/connector/instance.py", line 233, in _perform_refresh
    metadata = await metadata_task
               ^^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/ey/lib/python3.11/site-packages/google/cloud/sql/connector/client.py", line 124, in _get_metadata
    resp = await self._client.get(url, headers=headers, raise_for_status=True)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/ey/lib/python3.11/site-packages/aiohttp/client.py", line 693, in _request
    resp.raise_for_status()
  File "/home/avijendra/ey/lib

ClientResponseError: 404, message='Not Found', url=URL('https://sqladmin.googleapis.com/sql/v1beta4/projects/cdii-poc/instances/cdii-demo-temp/connectSettings')

['cdii-poc:us-central1:cdii-demo-temp']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "/home/avijendra/ey/lib/python3.11/site-packages/google/cloud/sql/connector/instance.py", line 298, in _refresh_task
    refresh_data = await refresh_task
                   ^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/ey/lib/python3.11/site-packages/google/cloud/sql/connector/instance.py", line 233, in _perform_refresh
    metadata = await metadata_task
               ^^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/ey/lib/python3.11/site-packages/google/cloud/sql/connector/client.py", line 124, in _get_metadata
    resp = await self._client.get(url, headers=headers, raise_for_status=True)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/ey/lib/python3.11/site-packages/aiohttp/client.py", line 693, in _request
    resp.raise_for_status()
  File "/home/avijendra/ey/lib

## Retrieve data 

In [8]:
# Retrieve the data from the PostgreDB and return a dataframe
def extract_data():
    tmp = pge.extract_data()
    df = DataFrame(tmp.fetchall())
    print(df)
    return df

df = extract_data()
df

    q_id                                           question  \
0    429  How many people are enrolled for Medi-Cal prog...   
1    430            How many of them live in Nevada County?   
2    431  How has participation in Medi-Cal progressed o...   
3    432  How does CalWorks program participation trends...   
4    433  In what way the race and ethnicity trends chan...   
5    434  Which county has the greatest proportion of FP...   
6    435  Which programs have the highest co-enrollment ...   
7    436  Which county has the lowest enrollment in WIC ...   
8    437  Which two counties have the highest number of ...   
9    438  How many Hispanic beneficiaries are being serv...   
10   439                   What is the breakdown by program   
11   440  Which counties have the highest and lowest rat...   

                                                  sql  \
0   SELECT  COALESCE(SUM(SAFE_CAST( Person AS INT6...   
1   SELECT SUM(CAST(Person AS INT64)) AS total_ben...   
2   selec

,q_id,question,sql,query_embedding
0,429,How many people are enrolled for Medi-Cal prog...,SELECT COALESCE(SUM(SAFE_CAST( Person AS INT6...,"[0.009485369548201561, 0.003129353281110525, -..."
1,430,How many of them live in Nevada County?,SELECT SUM(CAST(Person AS INT64)) AS total_ben...,"[-0.03890972211956978, -0.01862681470811367, -..."
2,431,How has participation in Medi-Cal progressed o...,"select year, sum(total_beneficiaries) as tota...","[0.05054508149623871, 0.001993116457015276, -0..."
3,432,How does CalWorks program participation trends...,SELECT COALESCE(SUM(SAFE_CAST( White AS INT6...,"[0.04451335594058037, -0.030554158613085747, -..."
4,433,In what way the race and ethnicity trends chan...,SELECT fileyear AS calhhs_dashboard_2015_2020...,"[0.04810802638530731, -0.012473777867853642, -..."
5,434,Which county has the greatest proportion of FP...,"SELECT Number AS county, COALESCE(SUM(SAFE_CAS...","[0.03396085649728775, -0.019975077360868454, -..."
6,435,Which programs have the highest co-enrollment ...,SELECT COALESCE(SUM(SAFE_CAST( Person AS INT...,"[0.04807215929031372, -0.01924615353345871, -0..."
7,436,Which county has the lowest enrollment in WIC ...,SELECT Vendor_Location AS wic_redemption_by_c...,"[0.050651632249355316, -0.04634733870625496, -..."
8,437,Which two counties have the highest number of ...,"SELECT Vendor_Location,(vendor_cnt/total_parti...","[0.030069099739193916, -0.05773473158478737, -..."
9,438,How many Hispanic beneficiaries are being serv...,SELECT COALESCE(SUM(SAFE_CAST( Hispanic AS ...,"[0.011656521819531918, 0.01710379682481289, -0..."


## Insert records to PostgreSQL and update index

In [ ]:
new_question = ""
mapped_sql = ""
pge.insert_row(new_question, mapped_sql)

## Search the VectorDB for closest matching queries

In [ ]:
# Search for the closest matching queries
new_query = 'What county has the highest number of beneficiaries enrolled in the WIC per capita?'
output_json = pge.search_matching_queries(new_query)
output_json

## Clear VectorDB and re-create VectorDB from all data in PostgreSQL

In [24]:
pge.recreate_vectordb_index()

Number of elements :  12


## Clear all Data in DB

In [ ]:
# pge.empty_table()
df = extract_data()
df